In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-18"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7101,2024-07-18,Eua Nba,20:00,Sacramento Kings,Washington Wizards,1.77,2.02,178.5,1.80,1.92,-2.5,1.99,2.28,foAfVAFr,0.564972,0.495050,0.555556,0.520833,0.060021,199.992,42.457286,0.212295,1.2,1.643168,1.369306,165.90,2.070,0.353058,0.170559,8.0,279.118,169.480281,0.607199,0.6,1.341641,2.236068,151.20,2.616,1.192091,0.455692,-26.0,105,80,1.58,1.89,212.180,532.442,0.093286,0.045620,0.096047,0.79,0.158,4.873418,0.604916,0.6,-0.004916,0.02,0.004,255.000000,0.309812,0.2,-0.109812
7102,2024-07-18,Eua Nba,21:00,Brooklyn Nets,Orlando Magic,2.79,1.43,179.5,1.89,1.89,0.0,0.00,0.00,2BD2Tlpe,0.358423,0.699301,0.529101,0.529101,0.057724,331.016,236.041786,0.713083,1.2,1.643168,1.369306,155.48,3.594,2.648694,0.736977,-25.0,181.160,40.275388,0.222319,1.8,1.643168,0.912871,211.68,1.844,0.428112,0.232165,21.0,92,98,1.69,2.16,228.624,238.948,0.455766,0.000000,NaN,3.24,0.648,2.762346,0.505604,0.6,0.094396,-3.63,-0.726,-0.592287,0.536038,0.3,-0.236038
7103,2024-07-18,Eua Nba,22:00,Cleveland Cavaliers,Los Angeles Lakers,1.43,2.79,178.5,1.89,1.89,0.0,0.00,0.00,dA6BRSE7,0.699301,0.358423,0.529101,0.529101,0.057724,372.492,351.013285,0.942338,0.6,1.341641,2.236068,169.15,3.864,3.524171,0.912052,-29.0,258.206,81.857667,0.317025,1.2,1.643168,1.369306,193.14,2.756,0.557611,0.202326,-23.0,85,87,1.99,2.22,244.546,287.086,0.455766,0.000000,NaN,-1.88,-0.376,-1.143617,0.580857,0.6,0.019143,1.47,0.294,6.088435,0.580218,0.7,0.119782
7104,2024-07-18,Eua Nba,23:00,Utah Jazz,Los Angeles Clippers,2.79,1.43,181.5,1.89,1.89,0.0,0.00,0.00,l4gckW6E,0.358423,0.699301,0.529101,0.529101,0.057724,290.680,214.643850,0.738420,1.8,1.643168,0.912871,183.18,2.850,1.683404,0.590668,-7.0,214.294,92.765345,0.432888,1.8,1.643168,0.912871,163.52,2.224,0.884098,0.397526,-9.0,86,112,2.13,1.46,367.208,269.812,0.455766,0.000000,NaN,-0.71,-0.142,-12.605634,0.399419,0.3,-0.099419,1.64,0.328,1.310976,0.578322,0.7,0.121678
7105,2024-07-18,Europa Eurobasket Sub,10:30,Republica Checa U20,Bélgica U20,2.76,1.41,137.5,1.82,1.88,0.0,0.00,0.00,0CEtIYXo,0.362319,0.709220,0.549451,0.531915,0.071539,462.812,280.960691,0.607073,0.6,1.341641,2.236068,900.00,6.844,3.779931,0.552299,-49.0,267.658,192.767525,0.720201,1.8,1.643168,0.912871,91.76,4.702,4.274994,0.909186,-15.0,72,74,12.50,1.24,0.000,0.000,0.457839,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7106,2024-07-18,México Lnbp,23:00,Halcones de Xalapa,Astros,1.94,1.78,171.5,1.85,1.85,0.0,0.00,0.00,tCm9IVyg,0.515464,0.561798,0.540541,0.540541,0.077262,280.170,53.226849,0.189981,0.6,1.341641,2.236068,253.30,3.552,0.633814,0.178439,-36.0,215.528,58.527092,0.271552,0.6,1.341641,2.236068,271.80,2.538,0.579025,0.228142,-23.0,85,90,2.98,3.02,0.000,236.610,0.060826,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,-0.50,-0.100,-7.800000,0.000000,0.0,0.000000
7107,2024-07-18,Europa Eurobasket Sub,10:30,Sérvia U20,Islândia U20,1.01,12.20,147.5,1.83,1.83,0.0,0.00,0.00,bqr7U2N8,0.990099,0.081967,0.546448,0.546448,0.072066,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,67,73,1.01,3.77,0.000,0.000,1.197960,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7108,2024-07-

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7102,21:00,Eua Nba,Brooklyn Nets,Orlando Magic,2.79,Back Home
